# INTEGO Prescriptions Cleaning: CNK to ATC mapping

## 1. Resources

Belgisch Centrum voor Farmacotherapeutische Informatie ([BCFI](https://www.bcfi.be/nl/)). Downloads available [here](https://www.bcfi.be/nl/download). Most recent file from "EMD databank" contains the full list of current medications. However, no mapping to ATC codes. Only to "HYR" system. HYR Classification is an internal hierarchical classification used at the BCFI for the layout of the booklet (Gecommentarieerd Geneesmiddelenrepertorium) and the BCFI website. This is not the same classification as the ATC classification system. This information can be found in the SAM2 [Logical Data Dossier](https://www.ehealth.fgov.be/ehealthplatform/file/cc73d96153bbd5448a56f19d925d05b1379c7f21/87d5d82491b214457ddfdd6b4a02d7714b19e767/sam2_-_logical_data_dossier_v2.3.pdf). SAM stand for "source authentique des médicaments”.

ATC coding system in maintained in SAM2 [database](https://www.vas.ehealth.fgov.be/websamcivics/samcivics/#). XML export can be downloaded from there. More information can be found in the SAM [portal](https://www.samportal.be/nl/sam).

## 2. Download XML file

Download the most recent file in the SAM2 [database](https://www.vas.ehealth.fgov.be/websamcivics/samcivics/#). This will download a folder named "sam-xxxx" with x an integer. 

This folder contains 8 files: 
- AMP-____.xml
- CMP-____.xml
- CPN-____.xml
- NONMEDICINAL-____.xml
- REF-____.xml
- RMB-____.xml
- RML-____.xml
- VMP-____.xml



The REF file conatains all the current ATC-codes. The AMP file should contain all CNK codes and ATC codes. You can for example also see how long certain products have been on the market.

For the purpose of this project, our primary focus is to extract the ATC and CNK codes belonging to the same medication. 

## 3. Extract ATC and CNK codes of medications from the AMP XML file

### 3.1. Parsing and exploring the XML file

The XML file contains lots or information about medications. We are mainly interested in ATC and CNK codes (and perhaps also the name) for mapping purposes within intego.

Resources on parsing XML files: 
- https://www.datacamp.com/tutorial/python-xml-elementtree
- https://stackoverflow.com/questions/64296527/parsing-nested-and-complex-xml-in-python

#### Parsing an xml file

In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as et 

Parse the XML file: read.

In [2]:
tree = et.parse('../intego_prescription_mapping/data/sam-8305/AMP-1709262051110.xml')
root = tree.getroot()

#### Initial exploration of XML file and how to work with it

In [3]:
root.tag

'{urn:be:fgov:ehealth:samws:v2:export}ExportActualMedicines'

In [4]:
root.attrib

{'version': '4.0', 'SamId': 'E.20240301_040048'}

Childs of the root are 'Amp' which has both or one of the following attributes 'code' and 'vmpCode'. Where the namespace is ns4 = urn:be:fgov:ehealth:samws:v2:export.

elem will recursively go over all children of root.

Ampp is child of Amp and has attribute 'ctiExtended'

Ampp has child 'Atc' that has attribute 'code'.

#### Define namespace

In [3]:
# Define Namespace
ns = {
    'xlmns':"urn:be:fgov:ehealth:samws:v2:actual:common",
    'ns2':"urn:be:fgov:ehealth:samws:v2:core",
    'ns3':"urn:be:fgov:ehealth:samws:v2:company:submit" ,
    'ns4':"urn:be:fgov:ehealth:samws:v2:export",
    'ns5':"urn:be:fgov:ehealth:samws:v2:refdata",
    'ns6':"urn:be:fgov:ehealth:samws:v2:consultation",
    'ns7':"urn:be:fgov:ehealth:samws:v2:reimbursement:submit",
    'ns8':"urn:be:fgov:ehealth:samws:v2:reimbursementlaw:submit",
    'ns9':"urn:be:fgov:ehealth:samws:v2:virtual:common",
    'ns10':"urn:be:fgov:ehealth:samws:v2:compounding:common",
    'ns11':"urn:be:fgov:ehealth:samws:v2:nonmedicinal:common",
    'ns12':"urn:be:fgov:ehealth:samws:v2:actual:status"
}

TEST: Code to extract date_from, date_to from each Ampp object. Even if one of the attributes is empty.

TEST code

In [12]:
results = []
for element in root.findall('ns4:Amp',ns):
    key_values = {}
    for key in ['vmpCode','code']:
        if key in element.attrib:
            key_values[key] = element.attrib[key]
        else:
            key_values[key] = None
            
            
    results.append([
        key_values.get('vmpCode',None),
        key_values.get('code', None)
    ])

### 3.2. Extract all ATC codes and CNK codes.

#### Generate lists of dictionaries 

In [4]:
results_atc = []
results_cnk = []
i = -1
for element in root.findall('ns4:Amp', ns):
    i = i+1
    for child_ampp in element.findall('ns4:Ampp',ns):
        for child_atc in child_ampp.findall('.//ns4:Atc',ns):
            results_atc.append(
                {
                'index':i,
                'atc':child_atc.attrib['code']
                }
                )
    
        for child_dmpp in child_ampp.findall('ns4:Dmpp',ns):
            results_cnk.append(
                {
                'index':i,
                'cnk':child_dmpp.attrib['code'],
                'type':child_dmpp.attrib['codeType']
                }
                )
    

            
     

#### Convert lists to data frames and save as parquet

In [43]:
df_atc = pd.DataFrame(results_atc)
df_cnk = pd.DataFrame(results_cnk)

In [44]:
import pyarrow as pa
import pyarrow.parquet as pq


table_atc = pa.Table.from_pandas(df_atc)
table_cnk = pa.Table.from_pandas(df_cnk)

pq.write_table(table_atc, 'atc.parquet')
pq.write_table(table_cnk, 'cnk.parquet')

#### Continue in R to merge the data frames into one final data frame. 

### 3.3. Extract additional information 


We can test the code with a smaller file: 

In [16]:
# Test with test file
tree = et.parse('../intego_prescription_mapping/data/test.xml')
root = tree.getroot()

#### 3.3.1. Get Data from "Data" elements as direct children of each "Amp" element

##### From the "Data" elements that are direct children of "Amp" element, we can extract:
- date_from: date from which the record was available. We can derive until when by ordering on this date.

And for each Data element with a certain date_from attribute: 
- OfficialName: Official name of the product 
- Status: If authorized
- PrescriptionNameFamph NL: Dutch specific name

Using elem.findtext('',namespaces=ns) is recommended becaus it returns None by default in case the element in of Noetype. The elem.find('').text would return an error. For a similar reason using elem.get('key') instead of elem.attrib['key']

##### Test the code 

In [101]:
results_data = []
i = -1

for element in root.findall('ns4:Amp', ns):
    i = i+1
    j = -1
    for child_data in element.findall('ns4:Data',ns):
        j=j+1
        print(i,
              j,
              element.get('vmpCode'),
              element.get('code'),
              #child_data.attrib['from'],
              child_data.get('from'),
              child_data.get('to'),
              #child_data.find('xlmns:OfficialName',ns).text,
              child_data.findtext('xlmns:OfficialName',namespaces=ns), # return None as default if NoneType
              child_data.findtext('xlmns:Status',namespaces=ns),
              child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns))
                

0 0 None SAM660978-00 2023-01-10 None Eupatorium Perfoliatum Boiron 5 CH AUTHORIZED Eupatorium Perfoliatum Boiron 5 CH globuli
1 0 16469 SAM000025-00 2017-09-19 2023-08-01 Haldol 5 mg/ml AUTHORIZED Haldol 5 mg/ml inj. opl. i.m. amp.
1 1 16469 SAM000025-00 1961-07-01 1961-07-01 Haldol 5 mg/ml AUTHORIZED None
1 2 16469 SAM000025-00 2023-08-02 None Haldol 5 mg/ml AUTHORIZED Haldol 5 mg/ml inj. opl. i.m. amp.
1 3 16469 SAM000025-00 1961-07-02 2016-09-30 Haldol 5 mg/ml AUTHORIZED Haldol 5 mg/ml inj. opl. i.m. amp.
1 4 16469 SAM000025-00 2016-10-01 2017-09-18 Haldol 5 mg/ml AUTHORIZED Haldol 5 mg/ml inj. opl. i.m. amp.


##### Extract the data in a dictionary 

In [106]:
results_data = []
#i = -1
for element in root.findall('ns4:Amp', ns):
    #i = i+1
    #j = -1
    for child_data in element.findall('ns4:Data',ns):
        #j=j+1
        results_data.append(
            {
                #'amp_index' :i,
                #'amp_data_index':j,
                'amp_vmpCode':element.get('vmpCode'),
                'amp_code':element.get('code'),
              #child_data.attrib['from'],
                'date_from':child_data.get('from'),
                'date_to':child_data.get('to'),
              #child_data.find('xlmns:OfficialName',ns).text,
                'official_name':child_data.findtext('xlmns:OfficialName',namespaces=ns), # return None as default if NoneType
                'status':child_data.findtext('xlmns:Status',namespaces=ns),
                'prescription_name_famph_nl':child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns),
                'prescription_name_nl':child_data.findtext('xlmns:PrescriptionName/ns2:Nl',namespaces=ns)
        }
        )
        
# Convert to data frame 
df_amp_data = pd.DataFrame(results_data)

In [107]:
df_amp_data

,amp_index,amp_data_index,amp_vmpCode,amp_code,date_from,date_to,official_name,status,prescription_name_famph_nl,prescription_name_nl
0,0,0,None,SAM660978-00,2023-01-10,None,Eupatorium Perfoliatum Boiron 5 CH,AUTHORIZED,Eupatorium Perfoliatum Boiron 5 CH globuli,None
1,1,0,16469,SAM000025-00,2017-09-19,2023-08-01,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,Haldol inj. oploss. i.m. [amp.] 5 mg / 1 ml
2,1,1,16469,SAM000025-00,1961-07-01,1961-07-01,Haldol 5 mg/ml,AUTHORIZED,None,None
3,1,2,16469,SAM000025-00,2023-08-02,None,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,Haldol inj. oploss. i.m. [amp.] 5 mg / 1 ml
4,1,3,16469,SAM000025-00,1961-07-02,2016-09-30,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,None
5,1,4,16469,SAM000025-00,2016-10-01,2017-09-18,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,Haldol inj. oploss. i.m./i.v. [amp.] 5 mg / 1 ml


#### 3.3.2. Get Data from "AmpComponent" elements as direct children of each "Amp" element

Not that important for now, skip this, and revisit when needed.

##### Test the code 

In [112]:

for element in root.findall('ns4:Amp', ns):

    for child_amp_component in element.findall('ns4:AmpComponent',ns):

        for child_data in child_amp_component.findall('ns4:Data',ns):
            print(
                  element.get('vmpCode'),
                  element.get('code'),
                  child_amp_component.get('vmpComponentCode'),
                  child_amp_component.get('sequenceNr'),
                  child_data.get('from'),
                  child_data.get('to'),
                  child_data.find('ns4:PharmaceuticalForm',ns).get('code'),
                  child_data.findtext('ns4:PharmaceuticalForm/ns5:Name/ns2:Nl',namespaces=ns),
                  child_data.find('ns4:RouteOfAdministration',ns).get('code'),
                  child_data.findtext('ns4:RouteOfAdministration/ns5:Name/ns2:Nl',namespaces=ns),
                  
                  child_data.findtext('xlmns:OfficialName',namespaces=ns), # return None as default if NoneType
                  child_data.findtext('xlmns:Status',namespaces=ns),
                  child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns))
            
        for child_actual_agent in child_amp_component.findall('ns4:RealActualIngredient',ns):
            #for elem in child_actual_agent.findall('ns4:Data',ns):
            print(
                child_actual_agent.get('rank')
            )
                

0 0 None SAM660978-00 None 1 2023-02-02 None 239 Globuli 72 Sublinguaal gebruik None None None
0 0 None SAM660978-00 None 1 2023-01-10 2023-02-01 239 Globuli 72 Sublinguaal gebruik None None None
1
1 0 16469 SAM000025-00 16469 1 2017-09-19 2023-08-01 306 Oplossing voor injectie 38 Intramusculair gebruik None None None
1 0 16469 SAM000025-00 16469 1 1961-07-01 1961-07-01 306 Oplossing voor injectie 38 Intramusculair gebruik None None None
1 0 16469 SAM000025-00 16469 1 2023-08-02 None 306 Oplossing voor injectie 38 Intramusculair gebruik None None None
1 0 16469 SAM000025-00 16469 1 1961-07-02 2016-09-30 306 Oplossing voor injectie 38 Intramusculair gebruik None None None
1 0 16469 SAM000025-00 16469 1 2016-10-01 2017-09-18 306 Oplossing voor injectie 38 Intramusculair gebruik None None None
1


#### 3.3.3. Get Data from "Ampp" elements as direct children of each "Amp" element
Important children of Ampp:
- Data: (sometimes) contains ATC code
- Dmpp: contains CNK code 
- AmppComponent (skip for now)

##### A. "Data" childs of "Ampp". theh contain information of ATC code.

###### Test the code 

In [150]:
i = -1
for element in root.findall('ns4:Amp', ns):
    i = i+1
    j = -1
    for child_ampp in element.findall('ns4:Ampp',ns):
        j=j+1
        for child_data in child_ampp.findall('ns4:Data',ns):
            if child_data.find('ns4:Atc',ns) is None: 
                atc_code = None
            else: 
                atc_code = child_data.find('ns4:Atc',ns).get('code')
            
            print(
                  element.get('vmpCode'),
                  element.get('code'),
                  child_ampp.get('ctiExtended'),
                  child_data.get('from'),
                  child_data.get('to'),
                  child_data.findtext('xlmns:Status',namespaces=ns),
                  child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns),
               
                  atc_code,
                  child_data.findtext('ns4:Atc/ns5:Description',namespaces=ns)
            )
            
                    
                
         

None SAM660978-00 660978-01 2023-01-10 2023-01-31 None None
None SAM660978-00 660978-01 2024-01-01 None None None
None SAM660978-00 660978-01 2023-02-01 2023-12-31 None None
16469 SAM000025-00 000025-01 1961-07-01 1961-07-01 N05AD01 Haloperidol
16469 SAM000025-00 000025-01 1961-07-02 None N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2020-07-02 2020-09-30 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2022-01-01 2022-05-31 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 1961-07-01 1961-07-01 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2022-06-01 2022-12-31 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2016-10-01 2017-06-12 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2020-10-01 2021-12-31 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2017-07-01 2017-09-18 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2024-01-01 None N05AD01 Haloperidol
16469 SAM000025-00 000025-02 1961-07-02 2016-09-30 N05AD01 Haloperidol
16469 SAM000025-00 000025-02 2023-01-01 2023-12-31 N05AD0

##### Extract the data in a dictionary 

In [152]:
results_dict = []

for element in root.findall('ns4:Amp', ns):
    for child_ampp in element.findall('ns4:Ampp',ns):
        for child_data in child_ampp.findall('ns4:Data',ns):
            if child_data.find('ns4:Atc',ns) is None: 
                atc_code = None
            else: 
                atc_code = child_data.find('ns4:Atc',ns).get('code')
            
            results_dict.append(
                {
                    'amp_vmpCode':element.get('vmpCode'),
                    'amp_code':element.get('code'),
                  'cti_extended':child_ampp.get('ctiExtended'),
                  'date_from':child_data.get('from'),
                  'date_to':child_data.get('to'),
                  'status':child_data.findtext('xlmns:Status',namespaces=ns),
                  'prescription_name_famph_nl':child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns),
                  'atc':atc_code,
                  'atc_description':child_data.findtext('ns4:Atc/ns5:Description',namespaces=ns)
                    
                }

            )
            
# Convert to data frame 
df_ampp = pd.DataFrame(results_dict)      
            

In [153]:
df_ampp

,amp_vmpCode,amp_code,cti_extended,date_from,date_to,atc,atc_description
0,None,SAM660978-00,660978-01,2023-01-10,2023-01-31,None,None
1,None,SAM660978-00,660978-01,2024-01-01,None,None,None
2,None,SAM660978-00,660978-01,2023-02-01,2023-12-31,None,None
3,16469,SAM000025-00,000025-01,1961-07-01,1961-07-01,N05AD01,Haloperidol
4,16469,SAM000025-00,000025-01,1961-07-02,None,N05AD01,Haloperidol
5,16469,SAM000025-00,000025-02,2020-07-02,2020-09-30,N05AD01,Haloperidol
6,16469,SAM000025-00,000025-02,2022-01-01,2022-05-31,N05AD01,Haloperidol
7,16469,SAM000025-00,000025-02,1961-07-01,1961-07-01,N05AD01,Haloperidol
8,16469,SAM000025-00,000025-02,2022-06-01,2022-12-31,N05AD01,Haloperidol
9,16469,SAM000025-00,000025-02,2016-10-01,2017-06-12,N05AD01,Haloperidol


##### B. "Dmpp" childs of "Ampp". They contain the CNK code 

###### Test code

In [151]:
i = -1
for element in root.findall('ns4:Amp', ns):
    i = i+1
    j = -1
    for child_ampp in element.findall('ns4:Ampp',ns):
        j=j+1
        for child_dmpp in child_ampp.findall('ns4:Dmpp',ns):
            
            print(
                  element.get('vmpCode'),
                  element.get('code'),
                  child_ampp.get('ctiExtended'),
                  child_dmpp.get('deliveryEnvironment'),
                  child_dmpp.get('code'),
                  child_dmpp.get('codeType'),
            )
            
                    
                
         

None SAM660978-00 660978-01 P 3104965 CNK
16469 SAM000025-00 000025-02 P 0046128 CNK
16469 SAM000025-00 000025-02 H 0708289 CNK
16469 SAM000025-00 000025-02 A 0708289 CNK


##### Extract the data in a dictionary 

In [154]:
results_dict = []
for element in root.findall('ns4:Amp', ns):
    for child_ampp in element.findall('ns4:Ampp',ns):

        for child_dmpp in child_ampp.findall('ns4:Dmpp',ns):
            
            results_dict.append(
                {
                  'amp_vmpCode':element.get('vmpCode'),
                  'amp_code':element.get('code'),
                  'cti_extended':child_ampp.get('ctiExtended'),
                  'delivery_env':child_dmpp.get('deliveryEnvironment'),
                  'cnk':child_dmpp.get('code'),
                  'code_type':child_dmpp.get('codeType')  
                }
                  
            )

# Convert to data frame 
df_dmpp = pd.DataFrame(results_dict) 

In [155]:
df_dmpp

,amp_vmpCode,amp_code,cti_extended,delivery_env,cnk,code_type
0,None,SAM660978-00,660978-01,P,3104965,CNK
1,16469,SAM000025-00,000025-02,P,0046128,CNK
2,16469,SAM000025-00,000025-02,H,0708289,CNK
3,16469,SAM000025-00,000025-02,A,0708289,CNK


##### C. "AmppComponent" childs of "Ampp" - SKIP for now.
Does not contain useful information for now.

In [ ]:
i = -1
for element in root.findall('ns4:Amp', ns):
    i = i+1
    j = -1
    for child_ampp in element.findall('ns4:Ampp',ns):
        j=j+1
        for child_data in child_ampp.findall('ns4:AmppComponent',ns):
            print(i,
                  j,
                  element.get('vmpCode'),
                  element.get('code'),
                  child_ampp.get('sequenceNr'),
                  child_data.get('from'),
                  child_data.get('to'),
                  child_data.findtext('xlmns:Status',namespaces=ns),
                  child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns))

                  child_data.findtext('xlmns:OfficialName',namespaces=ns), # return None as default if NoneType
            
        for child_actual_agent in child_ampp.findall('ns4:AmppComponent',ns):
            #for elem in child_actual_agent.findall('ns4:Data',ns):
            print(
                child_actual_agent.get('rank')
            )

### 3.4. Final extraction 
Final version of extracting ATC and CNK codes from this file as of 2024.04.10.

See section 3.3. for more explanation and test code. 

In [2]:
# Get the entire file 
tree = et.parse('../intego_prescription_mapping/data/sam-8305/AMP-1709262051110.xml')
root = tree.getroot()

In [4]:
# Define Namespace
ns = {
    'xlmns':"urn:be:fgov:ehealth:samws:v2:actual:common",
    'ns2':"urn:be:fgov:ehealth:samws:v2:core",
    'ns3':"urn:be:fgov:ehealth:samws:v2:company:submit" ,
    'ns4':"urn:be:fgov:ehealth:samws:v2:export",
    'ns5':"urn:be:fgov:ehealth:samws:v2:refdata",
    'ns6':"urn:be:fgov:ehealth:samws:v2:consultation",
    'ns7':"urn:be:fgov:ehealth:samws:v2:reimbursement:submit",
    'ns8':"urn:be:fgov:ehealth:samws:v2:reimbursementlaw:submit",
    'ns9':"urn:be:fgov:ehealth:samws:v2:virtual:common",
    'ns10':"urn:be:fgov:ehealth:samws:v2:compounding:common",
    'ns11':"urn:be:fgov:ehealth:samws:v2:nonmedicinal:common",
    'ns12':"urn:be:fgov:ehealth:samws:v2:actual:status"
}

#### 3.4.1. Get data from the direct "Data" children of "Amp"

In [5]:
results_data = []
for element in root.findall('ns4:Amp', ns):
    for child_data in element.findall('ns4:Data',ns):
        results_data.append(
            {

                'amp_vmpCode':element.get('vmpCode'),
                'amp_code':element.get('code'),
                'date_from':child_data.get('from'),
                'date_to':child_data.get('to'),
                'official_name':child_data.findtext('xlmns:OfficialName',namespaces=ns), # return None as default if NoneType
                'status':child_data.findtext('xlmns:Status',namespaces=ns),
                'prescription_name_famph_nl':child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns),
                'prescription_name_nl':child_data.findtext('xlmns:PrescriptionName/ns2:Nl',namespaces=ns)
        }
        )
        
# Convert to data frame 
df_amp_data = pd.DataFrame(results_data)

In [6]:
df_amp_data.head()

,amp_vmpCode,amp_code,date_from,date_to,official_name,status,prescription_name_famph_nl,prescription_name_nl
0,None,SAM660978-00,2023-01-10,None,Eupatorium Perfoliatum Boiron 5 CH,AUTHORIZED,Eupatorium Perfoliatum Boiron 5 CH globuli,None
1,16469,SAM000025-00,2017-09-19,2023-08-01,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,Haldol inj. oploss. i.m. [amp.] 5 mg / 1 ml
2,16469,SAM000025-00,1961-07-01,1961-07-01,Haldol 5 mg/ml,AUTHORIZED,None,None
3,16469,SAM000025-00,2023-08-02,None,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,Haldol inj. oploss. i.m. [amp.] 5 mg / 1 ml
4,16469,SAM000025-00,1961-07-02,2016-09-30,Haldol 5 mg/ml,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp.,None


#### 3.4.2. Get data from the direct "Ampp" children of "Amp"

Important children of Ampp:
- Data: (sometimes) contains ATC code
- Dmpp: contains CNK code 
- (AmppComponent (skip for now))

#### A. Data children: ATC

In [7]:
results_dict = []

for element in root.findall('ns4:Amp', ns):
    for child_ampp in element.findall('ns4:Ampp',ns):
        for child_data in child_ampp.findall('ns4:Data',ns):
            if child_data.find('ns4:Atc',ns) is None: 
                atc_code = None
            else: 
                atc_code = child_data.find('ns4:Atc',ns).get('code')
            
            results_dict.append(
                {
                    'amp_vmpCode':element.get('vmpCode'),
                    'amp_code':element.get('code'),
                  'cti_extended':child_ampp.get('ctiExtended'),
                  'date_from':child_data.get('from'),
                  'date_to':child_data.get('to'),
                  'status':child_data.findtext('xlmns:Status',namespaces=ns),
                  'prescription_name_famph_nl':child_data.findtext('xlmns:PrescriptionNameFamhp/ns2:Nl',namespaces=ns),
                  'atc':atc_code,
                  'atc_description':child_data.findtext('ns4:Atc/ns5:Description',namespaces=ns)
                    
                }

            )
            
# Convert to data frame 
df_ampp = pd.DataFrame(results_dict)      
            

In [8]:
df_ampp.head()

,amp_vmpCode,amp_code,cti_extended,date_from,date_to,status,prescription_name_famph_nl,atc,atc_description
0,None,SAM660978-00,660978-01,2023-01-10,2023-01-31,AUTHORIZED,Eupatorium Perfoliatum Boiron 5 CH globuli 4 g,None,None
1,None,SAM660978-00,660978-01,2024-01-01,None,AUTHORIZED,Eupatorium Perfoliatum Boiron 5 CH globuli 4 g,None,None
2,None,SAM660978-00,660978-01,2023-02-01,2023-12-31,AUTHORIZED,Eupatorium Perfoliatum Boiron 5 CH globuli 4 g,None,None
3,16469,SAM000025-00,000025-01,1961-07-01,1961-07-01,AUTHORIZED,None,N05AD01,Haloperidol
4,16469,SAM000025-00,000025-01,1961-07-02,None,AUTHORIZED,Haldol 5 mg/ml inj. opl. i.m. amp. 1 ml,N05AD01,Haloperidol


#### B. Dmpp children: CNK

In [9]:
results_dict = []
for element in root.findall('ns4:Amp', ns):
    for child_ampp in element.findall('ns4:Ampp',ns):

        for child_dmpp in child_ampp.findall('ns4:Dmpp',ns):
            
            results_dict.append(
                {
                  'amp_vmpCode':element.get('vmpCode'),
                  'amp_code':element.get('code'),
                  'cti_extended':child_ampp.get('ctiExtended'),
                  'delivery_env':child_dmpp.get('deliveryEnvironment'),
                  'cnk':child_dmpp.get('code'),
                  'code_type':child_dmpp.get('codeType')  
                }
                  
            )

# Convert to data frame 
df_dmpp = pd.DataFrame(results_dict) 

In [10]:
df_dmpp.head()

,amp_vmpCode,amp_code,cti_extended,delivery_env,cnk,code_type
0,None,SAM660978-00,660978-01,P,3104965,CNK
1,16469,SAM000025-00,000025-02,P,0046128,CNK
2,16469,SAM000025-00,000025-02,H,0708289,CNK
3,16469,SAM000025-00,000025-02,A,0708289,CNK
4,None,SAM660980-00,660980-01,P,3104981,CNK


#### 3.4.3. Save data frames as parquet files and merge further in R

In [11]:
import pyarrow as pa
import pyarrow.parquet as pq

In [12]:
table_data = pa.Table.from_pandas(df_amp_data)
table_ampp = pa.Table.from_pandas(df_ampp)
table_dmpp = pa.Table.from_pandas(df_dmpp)

pq.write_table(table_data, '../intego_prescription_mapping/amp_data.parquet')
pq.write_table(table_ampp, '../intego_prescription_mapping/ampp_data.parquet')
pq.write_table(table_dmpp, '../intego_prescription_mapping/ampp_dmpp.parquet')